In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
import geopandas as gpd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import plotly.express as px

In [ ]:
country_df = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

In [ ]:
country_df[((country_df['Date']=='2020-03-30')&(country_df['State/UnionTerritory']=='Tamil Nadu'))]

In [ ]:
#Copying the contry_df data to df for easy reference
df=country_df.copy()

In [ ]:
#Keeping it simple, we will drop the rows with state name ending with "***" as it is seems to be rows with incomplete values.
def drop_star(df):
    for i in df['State/UnionTerritory'].iteritems():
        if i[1][-3:] == "***":
            df.drop(i[0],inplace=True)
        
drop_star(df)
df['State/UnionTerritory'].unique()

In [ ]:
#lets convert the Date feature to Date&time datatype
df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d')

#Time is not required as it doesnt make much difference
df.drop(['Time'],axis=1, inplace=True)

#Renaming State/UnionTerritory to States for easy reference
df.rename(columns={'State/UnionTerritory':'States'}, inplace=True)

In [ ]:
df['Active_cases']=df['Confirmed']-(df['Cured']+df['Deaths'])
df['Discharge_Rate'] = np.round((df['Cured']/df['Confirmed'])*100, decimals = 4)      # create instances for 'death_rate and discharge_rate'
df['Death_Rate'] = np.round((df['Deaths']/df['Confirmed'])*100, decimals = 4)
df.head()

In [ ]:
df[df['States']=='Tamil Nadu'].tail()

In [ ]:
df[df['States']=='Maharashtra'].tail()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mtd
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
colors=['#0C68C7','#3A6794','#00FAF3','#FA643C','#C71D12']
sns.set(palette=colors, style='white')

sns.palplot(colors)

In [ ]:
#Current date from the record
current = df[df.Date == '2021-08-08']

In [ ]:
#Sorting data in descending ordrer (Confirmed)
max_confirmed_cases = current.sort_values(by='Confirmed', ascending = False)
max_confirmed_cases.head()

In [ ]:
#Selecting top 10 States/U.T
top_cases = max_confirmed_cases[:20]
top_cases.head()

In [ ]:
for feature in top_cases[['Confirmed','Cured','Deaths','Active_cases']]:
    fig=plt.figure(figsize=(30,10))
    plt.title("Top 10 highly impacted sates as on 7th June", size=10)
    ax=sns.barplot(data=top_cases,y=top_cases[feature],x='States', linewidth=0, edgecolor='black')
    plt.xlabel('States', size = 15)
    plt.ylabel(feature, size = 15)
    for i in ax.patches:
        ax.text(x=i.get_x(),y=i.get_height(),s=i.get_height())
plt.show()

In [ ]:
states=['Kerala', 'Tamil Nadu', 'Maharashtra', 'Tamil Nadu', 'Andhra Pradesh', 'Uttar Pradesh']
mh=df[df['States']=='Maharashtra']
kl=df[df['States']=='Kerala']
ka=df[df['States']=='Karnataka']
tn=df[df['States']=='Tamil Nadu']
ap=df[df['States']=='Andhra Pradesh']
up=df[df['States']=='Uttar Pradesh']

In [ ]:
fig, ax=plt.subplots(nrows=3, ncols=3, figsize=(23,10), squeeze=False, sharex=True, sharey=False, constrained_layout=True )
plt.suptitle("Comparison of Active, Cured & Deaths for top States", size = 25)
sns.lineplot(data=tn, x='Date',y='Active_cases', ax=ax[0,0], color=colors[1])
ax[0,0].set_title("Maharashtra", size=20)
sns.lineplot(data=tn, x='Date',y='Cured', ax=ax[1,0], color=colors[1])
sns.lineplot(data=tn, x='Date',y='Deaths', ax=ax[2,0], color=colors[1])

sns.lineplot(data=kl, x='Date',y='Active_cases', ax=ax[0,1], color=colors[2])
ax[0,1].set_title("Kerala", size=20)
sns.lineplot(data=kl, x='Date',y='Cured', ax=ax[1,1], color=colors[2])
sns.lineplot(data=kl, x='Date',y='Deaths', ax=ax[2,1], color=colors[2])

sns.lineplot(data=mh, x='Date',y='Active_cases', ax=ax[0,2], color=colors[3])
ax[0,2].set_title("Karnataka", size=20)
sns.lineplot(data=mh, x='Date',y='Cured', ax=ax[1,2], color=colors[3])
sns.lineplot(data=mh, x='Date',y='Deaths', ax=ax[2,2], color=colors[3])
plt.show()

In [ ]:
fig, ax=plt.subplots(nrows=3, ncols=3, figsize=(23,10), squeeze=False, sharex=True, sharey=False, constrained_layout=True )
sns.lineplot(data=tn, x='Date',y='Active_cases', ax=ax[0,0], color=colors[1])
ax[0,0].set_title("Tamil Nadu", size=20)
sns.lineplot(data=tn, x='Date',y='Cured', ax=ax[1,0], color=colors[1])
sns.lineplot(data=tn, x='Date',y='Deaths', ax=ax[2,0], color=colors[1])

sns.lineplot(data=kl, x='Date',y='Active_cases', ax=ax[0,1], color=colors[2])
ax[0,1].set_title("Andhra Pradesh", size=20)
sns.lineplot(data=kl, x='Date',y='Cured', ax=ax[1,1], color=colors[2])
sns.lineplot(data=kl, x='Date',y='Deaths', ax=ax[2,1], color=colors[2])

sns.lineplot(data=mh, x='Date',y='Active_cases', ax=ax[0,2], color=colors[3])
ax[0,2].set_title("Uttar Pradesh", size=20)
sns.lineplot(data=mh, x='Date',y='Cured', ax=ax[1,2], color=colors[3])
sns.lineplot(data=mh, x='Date',y='Deaths', ax=ax[2,2], color=colors[3])
plt.show()

In [ ]:
df['Date']= pd.to_datetime(df['Date'])          # Date is converted to DateTime format.
data_20 = df[df['Date'].dt.year==2020]               # Considering data of only the year 2020.   
data_21 = df[df['Date'].dt.year==2021]               # Considering data of only the year 2021.  

data_20['Month']=data_20['Date'].dt.month                    # Month is accessed from the DateTime object.
data_21['Month']=data_21['Date'].dt.month    

#Year 2020
data_confirm_20= data_20['Confirmed'].groupby(data_20['Month']).sum()
data_dis_20= data_20['Cured'].groupby(data_20['Month']).sum()      # creating instances for 'confirmed','deaths','discharged' by month column
data_death_20= data_20['Deaths'].groupby(data_20['Month']).sum()

#Year 2021
data_confirm_21= data_21['Confirmed'].groupby(data_21['Month']).sum()
data_dis_21= data_21['Cured'].groupby(data_21['Month']).sum()      # creating instances for 'confirmed','deaths','discharged' by month column
data_death_21= data_21['Deaths'].groupby(data_21['Month']).sum()

cols_20=[data_confirm_20,data_dis_20,data_death_20]
data_20=pd.concat(cols_20,axis=1)  

cols_21=[data_confirm_21,data_dis_21,data_death_21]
data_21=pd.concat(cols_21,axis=1) 

#Year 2020
data_20['discharge_rate_20'] = np.round((data_20['Cured']/data_20['Confirmed'])*100, decimals=4)      # create instances for 'death_rate and discharge_rate'
data_20['death_rate_20'] = np.round((data_20['Deaths']/data_20['Confirmed'])*100, decimals=4)

#Year 2020
data_21['discharge_rate_21'] = np.round((data_21['Cured']/data_21['Confirmed'])*100, decimals=4)      # create instances for 'death_rate and discharge_rate'
data_21['death_rate_21'] = np.round((data_21['Deaths']/data_21['Confirmed'])*100, decimals=4)

In [ ]:
#Year 2020
data_20.reset_index(inplace=True)
data_20.head()

In [ ]:
#Year 2021
data_21.reset_index(inplace=True)
data_21.head()

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x="Month",y="discharge_rate_20",data=data_20,color="g",lw=3,marker='o',markersize=10)
plt.title('DISCHARGE RATE PER MONTH IN 2020')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x="Month",y="discharge_rate_21",data=data_21,color="g",lw=3,marker='o',markersize=10)
plt.title('DISCHARGE RATE PER MONTH IN 2021')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x="Month",y="death_rate_20",data=data_20,color="r",lw=3,marker='o',markersize=10)
plt.title('DEATH RATE PER MONTH IN 2020')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x="Month",y="death_rate_21",data=data_21,color="r",lw=3,marker='o',markersize=10)
plt.title('DEATH RATE PER MONTH IN 2021')
plt.show()

In [ ]:
data_20['Active_cases']=data_20['Confirmed']-(data_20['Cured']+data_20['Deaths'])
plt.figure(figsize=(10,5))
sns.lineplot(x="Month",y="Active_cases",data=data_20,color="b",lw=3,marker='o',markersize=10)
plt.title('DEATH RATE PER MONTH IN 2020')
plt.show()

In [ ]:
tn=df[df['States']=='Tamil Nadu']['Cured']
mh=df[df['States']=='Maharashtra']['Cured']
kl=df[df['States']=='Kerala']['Cured']

In [ ]:
top_cases.head()

In [ ]:
top_cases['Fatality-Ratio'] = top_cases['Deaths']/top_cases['Confirmed']
a4_dims = (15, 7)
fig, ax = plt.subplots(figsize=a4_dims)
plt.title('Fatality-Ratio of Top 10 Contaminated States',size=15)
sns.pointplot(data=top_cases,x='States',y='Fatality-Ratio',ax=ax,color='Red')
plt.show()

In [ ]:
top_cases['Recovery-Ratio'] = top_cases['Cured']/top_cases['Confirmed']
a4_dims = (15, 7)
fig, ax = plt.subplots(figsize=a4_dims)
plt.title('Recovery-Ratio of Top 10 Contaminated States',size=15)
sns.pointplot(data=top_cases,x='States',y='Recovery-Ratio',ax=ax,color='Green')
plt.show()

In [ ]:
df.head()

In [ ]:
print('Total number of Active Covid-19 cases across India : {}'.format(max_confirmed_cases['Active_cases'].sum()))
top_cases = max_confirmed_cases.groupby('States')['Active_cases'].max().sort_values(ascending=False).to_frame()
top_cases.style.background_gradient(cmap='flare')

In [ ]:
from fbprophet import Prophet
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
recovered = df.groupby('Date').sum()['Cured'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
confirmed.head()

In [ ]:
confirmed.columns = ['ds', 'y']
confirmed['ds'] = pd.to_datetime(confirmed['ds'])

In [ ]:
m=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True, changepoint_prior_scale=2)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5, prior_scale=0.02)
m.add_country_holidays(country_name='India')
m.fit(confirmed)
future = m.make_future_dataframe(periods=50, freq='D')       #Making future prediction for next 7 days
future.tail(10)

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(30)

In [ ]:
confirmed_forecast_plot = m.plot(forecast)

In [ ]:
confirmed_forecast_plot = m.plot_components(forecast)

In [ ]:
m.train_holiday_names.to_list()

In [ ]:
print('Average recovery rate Covid-19 cases across India : {}'.format(max_confirmed_cases['Discharge_Rate'].mean()))
top_cases = max_confirmed_cases.groupby('States')['Discharge_Rate'].max().sort_values(ascending=False).to_frame()
top_cases.style.background_gradient(cmap='flare')

In [ ]:
discharge_rate = df[['Date','Discharge_Rate']]
discharge_rate.head()

In [ ]:
discharge_rate.columns = ['ds', 'y']
discharge_rate['ds'] = pd.to_datetime(discharge_rate['ds'])

In [ ]:
m = Prophet(interval_width=0.90)
m.fit(discharge_rate)
future = m.make_future_dataframe(periods=7)       #Making future prediction for next 7 days
future.tail(10)

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
confirmed_forecast_plot = m.plot(forecast)

In [ ]:
confirmed_forecast_plot = m.plot_components(forecast)

In [ ]:
print('Average recovery rate Covid-19 cases across India : {}'.format(max_confirmed_cases['Discharge_Rate'].mean()))
top_cases = max_confirmed_cases.groupby('States')['Death_Rate'].max().sort_values(ascending=False).to_frame()
top_cases.style.background_gradient(cmap='flare')

In [ ]:
discharge_rate = df[['Date','Death_Rate']]
discharge_rate.head()

In [ ]:
discharge_rate.columns = ['ds', 'y']
discharge_rate['ds'] = pd.to_datetime(discharge_rate['ds'])

In [ ]:
m = Prophet(interval_width=0.50)
m.fit(discharge_rate)
future = m.make_future_dataframe(periods=7)       #Making future prediction for next 7 days
future.tail(10)

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
top_cases = max_confirmed_cases.groupby('States')['Deaths'].max().sort_values(ascending=False).to_frame()
top_cases.style.background_gradient(cmap='flare')

In [ ]:
deaths = df[['Date','Deaths']]
deaths.head()

In [ ]:
deaths.columns = ['ds', 'y']
deaths['ds'] = pd.to_datetime(deaths['ds'])

In [ ]:
m=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True, changepoint_prior_scale=2)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5, prior_scale=0.02)
m.add_country_holidays(country_name='India')
m.fit(confirmed)
future = m.make_future_dataframe(periods=50, freq='D')       #Making future prediction for next 7 days
future.tail(10)

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
confirmed_forecast_plot = m.plot_components(forecast)

In [ ]:
df.head()

In [ ]:
#Summarizing the analysis of all the states
#State-wise Report
state_cases = max_confirmed_cases.drop(['Sno','Date','ConfirmedIndianNational',
                                       'ConfirmedForeignNational'],axis=1)

state_wise_cases = state_cases.sort_values('Confirmed', ascending= False).fillna(0).style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Cured"])\
                        .background_gradient(cmap='Oranges',subset=["Active_cases"])\
                        .background_gradient(cmap='RdYlBu',subset=["Death_Rate"])\
                        .background_gradient(cmap='Accent',subset=["Discharge_Rate"])

state_wise_cases

In [ ]:
#Loading the vaccination data
vaccine_df = pd.read_csv('../input/covid19-in-india/covid_vaccine_statewise.csv')

In [ ]:
vaccine_df.head()

In [ ]:
vacc_df=vaccine_df.copy()
vaccine_df = vaccine_df[['Updated On','State','Total Doses Administered','First Dose Administered','Second Dose Administered',
                         'Male (Doses Administered)','Female (Doses Administered)','Transgender (Doses Administered)',
                         '18-44 Years (Doses Administered)','45-60 Years (Doses Administered)','60+ Years (Doses Administered)',
                         '18-44 Years(Individuals Vaccinated)','45-60 Years(Individuals Vaccinated)','60+ Years(Individuals Vaccinated)',
                         'Male(Individuals Vaccinated)','Female(Individuals Vaccinated)','Transgender(Individuals Vaccinated)',
                         'Total Individuals Vaccinated']]
vaccine_df.columns = ['Date','States','Total Doses Administered','First Dose Administered','Second Dose Administered',
                      'Male (Doses Administered)','Female (Doses Administered)','Transgender (Doses Administered)',
                      '18-44 Years (Doses Administered)','45-60 Years (Doses Administered)','60+ Years (Doses Administered)',
                      '18-44 Years(Individuals Vaccinated)','45-60 Years(Individuals Vaccinated)','60+ Years(Individuals Vaccinated)',
                      'Male Vaccinated','Female Vaccinated','Transgender Vaccinated','Total Individuals Vaccinated']
vaccine_df.head()

In [ ]:
vaccine_df['Date'] = pd.to_datetime(vaccine_df['Date'], format = '%d/%m/%Y')
latest_date = max(vaccine_df['Date'])
print("Current Date : ", latest_date)

In [ ]:
male = vaccine_df["Male Vaccinated"].max() 
female = vaccine_df["Female Vaccinated"].max()  
trans = vaccine_df["Transgender Vaccinated"].max()
px.pie(names=["Male Vaccinated","Female Vaccinated","Trans Gender"],values=[male,female,trans],title="Male, Female and Transgender Vaccinated ratio for Covid19",width=550,height=400)

In [ ]:
Doses = vaccine_df["Total Doses Administered"].max() 
Vaccinated = vaccine_df["Total Individuals Vaccinated"].max()  
px.pie(names=["Doses Administered","People Vaccinated"],values=[Doses,Vaccinated],title="Doses administered vs People Vaccinated",width=500,height=400)

In [ ]:
vaccine_18_44 = vaccine_df['18-44 Years(Individuals Vaccinated)'].max() 
vaccine_44_60 = vaccine_df['45-60 Years(Individuals Vaccinated)'].max()  
vaccine_60_plus = vaccine_df['60+ Years(Individuals Vaccinated)'].max() 
px.pie(names=["vaccine_18_44","vaccine_44_60","vaccine_60_plus"],values=[vaccine_18_44,vaccine_44_60,vaccine_60_plus],title="Age-wise vaccinated",width=500,height=400)

In [ ]:
vaccine_df['Net Vaccinated']=vaccine_df['18-44 Years(Individuals Vaccinated)'] + vaccine_df['45-60 Years(Individuals Vaccinated)'] + vaccine_df['60+ Years(Individuals Vaccinated)']

In [ ]:
vaccine = vaccine_df[['Date','States','Net Vaccinated']]
vaccine.dropna(inplace=True)
vaccine

In [ ]:
#Current date from the record
current = vaccine[vaccine.Date == '2021-06-24']

In [ ]:
#Sorting data in descending ordrer (Confirmed)
max_confirmed_cases = current.sort_values(by='Net Vaccinated', ascending = False)
max_confirmed_cases

In [ ]:
vaccine['Date'] = pd.to_datetime(vaccine['Date'], format = '%d/%m/%Y')
latest_date = max(vaccine['Date'])
print("Current Date : ", latest_date)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=100, figsize=(6,4))

sns.lineplot(data=vaccine_df[vaccine_df['States']=='India'], x=vaccine_df['Date'], y='Total Individuals Vaccinated', ax=ax)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(DateFormatter('%b%y'))
ax.set_ylabel('Total Indians Vaccinated so far in millions', fontsize=10)
ax.set_yticklabels([0,0,20,40,60,80,100,120])

Xstart, Xend = ax.get_xlim()
plt.title('India\'s Vaccination Performance')
plt.show()

In [ ]:
#Current date from the record
current = vaccine_df[vaccine_df.Date == '2021-08-08']

In [ ]:
vaccine_df[vaccine_df['States']=='India'].max()

In [ ]:
#Sorting data in descending ordrer (Confirmed)
max_confirmed_cases = current.sort_values(by='Total Individuals Vaccinated', ascending = False)
max_confirmed_cases

In [ ]:
Vaccinated